In [1]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID").filter(regex= "EEG.*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [3]:
# Load the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.values, labels["TARGET"].values, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().partial_fit(X_train)
scaler = StandardScaler().partial_fit(X_val)
scaler = StandardScaler().partial_fit(X_test)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(None,1, X_train.shape[1]),
                                        input_var=input_var)
    network = lasagne.layers.Conv1DLayer(
        network, num_filters=8, filter_size=50,
        nonlinearity=lasagne.nonlinearities.rectify,
        #W=lasagne.init.GlorotUniform())
        )
    network = lasagne.layers.MaxPool1DLayer(network, pool_size=10)
    #network = lasagne.layers.Conv2DLayer(
    #    network, num_filters=32, filter_size=(10, 1),
    #    nonlinearity=lasagne.nonlinearities.rectify,
    #    W=lasagne.init.GlorotUniform())
    #network = lasagne.layers.MaxPool2DLayer(network, pool_size=(5, 1))
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.25),
        num_units=1000,
        nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.25),
        num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [6]:
#%pdb

In [7]:
# Prepare Theano variables for inputs and targets
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = build_cnn(input_var)

In [8]:
X_train=X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_val=X_val.reshape(X_val.shape[0],1,X_val.shape[1])

In [9]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [10]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.0005, momentum=0.9)

In [11]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

In [12]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [13]:
from tempfile import mkstemp

In [14]:
#tempfile.mkstemp(dir="../tmp")

In [15]:
#import os
#os.environ["THEANO_FLAGS"] = "blas.ldflags=-llapack -lblas -lgfortran"

In [16]:
train_fn =theano.function([input_var, target_var], loss, updates=updates)

In [17]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]



In [ ]:
import time
num_epochs = 20
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 64, shuffle=True):
        inputs, targets = batch
        targets = targets.astype(np.int32)
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 64, shuffle=False):
        inputs, targets = batch
        targets = targets.astype(np.int32)
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 20 took 210.931s
  training loss:		1.225793
  validation loss:		1.049981
  validation accuracy:		54.76 %
Epoch 2 of 20 took 204.638s
  training loss:		1.013255
  validation loss:		0.935014
  validation accuracy:		62.65 %
Epoch 3 of 20 took 214.396s
  training loss:		1.033490
  validation loss:		1.132009
  validation accuracy:		62.50 %
Epoch 4 of 20 took 225.974s
  training loss:		0.928381
  validation loss:		0.908911
  validation accuracy:		66.46 %
Epoch 5 of 20 took 206.785s
  training loss:		0.862250
  validation loss:		0.838830
  validation accuracy:		68.13 %
Epoch 6 of 20 took 219.566s
  training loss:		0.834610
  validation loss:		0.829352
  validation accuracy:		68.17 %
Epoch 7 of 20 took 213.341s
  training loss:		0.817338
  validation loss:		0.840410
  validation accuracy:		67.66 %
Epoch 8 of 20 took 204.577s
  training loss:		0.797099
  validation loss:		0.796440
  validation accuracy:		69.08 %
Epoch 9 of 20 took 201.646s
  training loss:		0.779828
  validation loss

In [17]:
X_train

array([[ -3.27090272,  -5.17772412,  -5.93906008, ...,  -4.48844825,
         -6.07158702,  -6.32548659],
       [  0.81069452,   0.47279711,   0.20199726, ..., -36.21900083,
        -34.24005984, -33.34316947],
       [-21.6893264 , -19.19169591, -18.05410131, ..., -31.74934691,
        -32.01672954, -31.93865097],
       ..., 
       [ 23.2823655 ,  22.67404351,  21.55815244, ...,  17.25645074,
         14.23969712,   9.22338637],
       [ -9.58074305,  -6.7713347 ,  -6.12195638, ...,   7.8565183 ,
         11.19261089,  13.00891586],
       [ -0.92509361,  -0.31970086,   0.12211495, ...,   0.88637613,
          0.5123374 ,  -0.10109219]])

In [26]:
from sklearn.metrics import accuracy_score
predicted_values = test_prediction.flatten()
print "test loss:", test_loss(predicted_values,y_test)

test loss:

TypeError: 'TensorVariable' object is not callable

In [27]:
predicted_values

Reshape{1}.0